# Notebook that loads a Filtered Signal and Exports it into a format that can be used by the remaining Signal-to-Spike Stages

### Check WD (change if necessary) and file loading

In [403]:
# Show current directory
import os
curr_dir = os.getcwd()
print(curr_dir)

# Check if the current WD is the file location
if "/src/hfo/signal_to_spike" not in os.getcwd():
    # Set working directory to this file location
    file_location = f"{os.getcwd()}/thesis-lava/src/hfo/signal_to_spike"
    print("File Location: ", file_location)

    # Change the current working Directory
    os.chdir(file_location)

    # New Working Directory
    print("New Working Directory: ", os.getcwd())

PATH_TO_FILE = '' # 'src/hfo/'  # This is needed if the WD is not the same as the file location

/home/monkin/Desktop/feup/thesis/thesis-lava/src/hfo/signal_to_spike


## Extract the channel for the 3 frequency bands

In [404]:
from utils.input import RIPPLE_BAND_FILENAME, FR_BAND_FILENAME, BOTH_BAND_FILENAME

ripple_file_suffix = f'{RIPPLE_BAND_FILENAME}_band'
fr_file_suffix = f'{FR_BAND_FILENAME}_band'
hfo_file_suffix = f'{BOTH_BAND_FILENAME}_band'

In [405]:
import numpy as np
import math
from utils.io import preview_np_array

IS_CLINICAL = True
INPUT_PATH = "/clinical" if IS_CLINICAL else "/synthetic"
INPUT_FILE_NAME = "filtered_seeg_ics_ch30-59"   # "filtered_seeg_csl_ch30-59"   # f"filtered_seeg_ch210-239"    # f"filtered_seeg_ch90-119"     # 

filtered_seeg_path = f"../filter/results{INPUT_PATH}/{INPUT_FILE_NAME}"

ripple_seeg_filename = f"{filtered_seeg_path}_{ripple_file_suffix}.npy"
fr_seeg_filename = f"{filtered_seeg_path}_{fr_file_suffix}.npy"
hfo_seeg_filename = f"{filtered_seeg_path}_{hfo_file_suffix}.npy"

ripple_data = np.load(f"{PATH_TO_FILE}{ripple_seeg_filename}")
fr_data = np.load(f"{PATH_TO_FILE}{fr_seeg_filename}")
hfo_data = np.load(f"{PATH_TO_FILE}{hfo_seeg_filename}")

In [406]:
preview_np_array(ripple_data, "ripple_data", edge_items=2)

preview_np_array(fr_data, "fr_data", edge_items=2)

preview_np_array(hfo_data, "hfo_data", edge_items=2)

ripple_data Shape: (125056, 30).
Preview: [[-3.45583186e-05 -7.22943633e-05 ... -2.10924785e-04  2.39127537e-04]
 [-5.05349660e-04 -1.05106314e-03 ... -3.07229283e-03  3.48499665e-03]
 ...
 [ 1.20587999e+00 -6.45394297e-01 ... -2.85881185e-01  3.25530382e+00]
 [ 5.39099496e-01 -5.29644202e-01 ... -8.25277516e-01  3.60921031e+00]]
fr_data Shape: (125056, 30).
Preview: [[-6.68777581e-04 -1.39905098e-03 ... -4.08184695e-03  4.62763070e-03]
 [-4.69889710e-03 -9.71173697e-03 ... -2.84456680e-02  3.22860535e-02]
 ...
 [ 8.15717077e-01 -4.35934484e-01 ...  3.14394276e-01 -9.11249438e-01]
 [-7.03969363e-01 -3.12499050e-01 ...  4.60469373e-01  9.18939256e-01]]
hfo_data Shape: (125056, 30).
Preview: [[-0.02945923 -0.0616273  ... -0.17980275  0.20384418]
 [-0.2769466  -0.57415547 ... -1.68002993  1.90629282]
 ...
 [-0.45465793  0.52715916 ... -0.93174099  1.92176336]
 [ 0.31837213  0.01130678 ...  0.29668694  2.4572324 ]]


## Define Global Parameters of the Experiment

In [407]:
from utils.input import SAMPLING_RATE, X_STEP

num_samples = ripple_data.shape[0]    # 2048 * 120 = 245760
num_channels = ripple_data.shape[1]   # 960

print(f"num_samples: {num_samples}, num_channels: {num_channels}")

num_samples: 125056, num_channels: 30


## Define the Channel we want to extract to feed the SNN

In [408]:
ch_idx = 26

## Fetch the signal for the specified channel

In [409]:
# Extract the seeg_channels in the range [ch_start_idx, ch_end_idx]
ripple_seeg_ch = ripple_data[:, ch_idx]
fr_seeg_ch = fr_data[:, ch_idx]
hfo_seeg_ch = hfo_data[:, ch_idx]

preview_np_array(ripple_seeg_ch, "Ripple sEEG Channel")
preview_np_array(fr_seeg_ch, "FR sEEG Channel")
preview_np_array(hfo_seeg_ch, "HFO sEEG Channel")

Ripple sEEG Channel Shape: (125056,).
Preview: [ 1.01688786e-04  1.48312419e-03  1.03505373e-02  4.59330006e-02
  1.44846460e-01 ...  2.79686384e+00  2.08877449e+00  1.23923651e+00
  3.91468362e-01 -3.35923047e-01]
FR sEEG Channel Shape: (125056,).
Preview: [ 0.0019679   0.01375153  0.02591011 -0.04557574 -0.24900832 ...
  0.40517923  0.04589511 -0.25397784 -0.85574955 -1.15821117]
HFO sEEG Channel Shape: (125056,).
Preview: [ 0.08668457  0.81161464  3.1972572   6.36634517  4.56066843 ...
  1.46877805 -0.37909596 -0.66129324 -0.28660881 -0.62609222]


## Import the Markers (Annotated Events) 
The markers are stored in a numpy array of shape (num_channels, events):
- Each row represents the events of a channel
- Each event is composed of the following 3 fields (Label, Position, Shape)

In [410]:
markers_seeg_file_name = f"{filtered_seeg_path}_markers.npy"
markers = np.load(f"{PATH_TO_FILE}{markers_seeg_file_name}", allow_pickle=True)

print("Markers shape: ", markers.shape)
print("First time steps: ", markers[:10])

Markers shape:  (30,)
First time steps:  [array([('Ripple',  2598.6328, 0.), ('Ripple', 32159.18  , 0.),
        ('Ripple', 35217.773 , 0.), ('Ripple', 49569.824 , 0.)],
       dtype=[('label', '<U64'), ('position', '<f4'), ('duration', '<f4')])
 array([('Ripple',  2598.6328, 0.), ('Ripple', 32159.18  , 0.),
        ('Ripple', 35217.773 , 0.), ('Ripple', 49569.824 , 0.)],
       dtype=[('label', '<U64'), ('position', '<f4'), ('duration', '<f4')])
 array([('Ripple',  2598.6328, 0.), ('Ripple', 32159.18  , 0.),
        ('Ripple', 35217.773 , 0.)],
       dtype=[('label', '<U64'), ('position', '<f4'), ('duration', '<f4')])
 list([array([],
       dtype=[('label', '<U64'), ('position', '<f4'), ('duration', '<f4')])])
 list([array([],
       dtype=[('label', '<U64'), ('position', '<f4'), ('duration', '<f4')])])
 list([array([],
       dtype=[('label', '<U64'), ('position', '<f4'), ('duration', '<f4')])])
 list([array([],
       dtype=[('label', '<U64'), ('position', '<f4'), ('duration', '<f

### Get the markers of the specified channel

In [411]:
relevant_markers = markers[ch_idx]

preview_np_array(relevant_markers, "relevant_markers", edge_items=10)

relevant_markers Shape: (43,).
Preview: [('Fast Ripple',  3478.5156, 0.) ('Ripple',  4852.539 , 0.)
 ('Ripple',  5303.711 , 0.) ('Ripple',  9606.445 , 0.)
 ('Fast Ripple',  9820.3125, 0.) ('Fast Ripple', 11166.016 , 0.)
 ('Fast Ripple', 18351.562 , 0.) ('Fast Ripple', 18539.062 , 0.)
 ('Fast Ripple', 23125.488 , 0.) ('Fast Ripple', 24826.66  , 0.) ...
 ('Fast Ripple', 50780.76  , 0.) ('Ripple', 51099.12  , 0.)
 ('Fast Ripple', 51099.12  , 0.) ('Fast Ripple', 51583.496 , 0.)
 ('Fast Ripple', 53103.027 , 0.) ('Fast Ripple', 53792.48  , 0.)
 ('Fast Ripple', 55280.76  , 0.) ('Fast Ripple', 56544.434 , 0.)
 ('Fast Ripple', 56882.324 , 0.) ('Fast Ripple', 58532.715 , 0.)]


## Define the Markers of each Band

In [412]:
from utils.input import label_has_fast_ripple, label_has_ripple, label_has_hfo_event

ripple_markers = []
fr_markers = []
hfo_markers = []

for marker in relevant_markers:
    curr_label = marker['label']
    if label_has_ripple(curr_label):
        ripple_markers.append(marker)
    if label_has_fast_ripple(curr_label):
        fr_markers.append(marker)
    if label_has_hfo_event(curr_label):
        hfo_markers.append(marker)

ripple_markers = np.array(ripple_markers)
fr_markers = np.array(fr_markers)
hfo_markers = np.array(hfo_markers)

In [413]:
preview_np_array(ripple_markers, "ripple_markers", edge_items=4)
preview_np_array(fr_markers, "fr_markers", edge_items=4)
preview_np_array(hfo_markers, "hfo_markers", edge_items=4)

ripple_markers Shape: (7,).
Preview: [('Ripple',  4852.539, 0.) ('Ripple',  5303.711, 0.)
 ('Ripple',  9606.445, 0.) ('Ripple', 32159.18 , 0.)
 ('Ripple', 34225.586, 0.) ('Ripple', 38057.13 , 0.)
 ('Ripple', 51099.12 , 0.)]
fr_markers Shape: (36,).
Preview: [('Fast Ripple',  3478.5156, 0.) ('Fast Ripple',  9820.3125, 0.)
 ('Fast Ripple', 11166.016 , 0.) ('Fast Ripple', 18351.562 , 0.) ...
 ('Fast Ripple', 55280.76  , 0.) ('Fast Ripple', 56544.434 , 0.)
 ('Fast Ripple', 56882.324 , 0.) ('Fast Ripple', 58532.715 , 0.)]
hfo_markers Shape: (43,).
Preview: [('Fast Ripple',  3478.5156, 0.) ('Ripple',  4852.539 , 0.)
 ('Ripple',  5303.711 , 0.) ('Ripple',  9606.445 , 0.) ...
 ('Fast Ripple', 55280.76  , 0.) ('Fast Ripple', 56544.434 , 0.)
 ('Fast Ripple', 56882.324 , 0.) ('Fast Ripple', 58532.715 , 0.)]


## Export the Processed sEEG and respective markers into a `.npy` file

In [414]:
# To keep the same format as the original data, we will add an extra dimension to the final markers
final_ripple_markers = np.array([ripple_markers])
final_fr_markers = np.array([fr_markers])
final_hfo_markers = np.array([hfo_markers])

preview_np_array(final_ripple_markers, "final_ripple_markers", edge_items=2)
preview_np_array(final_fr_markers, "final_fr_markers", edge_items=2)
preview_np_array(final_hfo_markers, "final_hfo_markers", edge_items=2)

final_ripple_markers Shape: (1, 7).
Preview: [[('Ripple',  4852.539, 0.) ('Ripple',  5303.711, 0.) ...
  ('Ripple', 38057.13 , 0.) ('Ripple', 51099.12 , 0.)]]
final_fr_markers Shape: (1, 36).
Preview: [[('Fast Ripple',  3478.5156, 0.) ('Fast Ripple',  9820.3125, 0.) ...
  ('Fast Ripple', 56882.324 , 0.) ('Fast Ripple', 58532.715 , 0.)]]
final_hfo_markers Shape: (1, 43).
Preview: [[('Fast Ripple',  3478.5156, 0.) ('Ripple',  4852.539 , 0.) ...
  ('Fast Ripple', 56882.324 , 0.) ('Fast Ripple', 58532.715 , 0.)]]


In [415]:
EXPORT_MARKERS = True

# Define the parent folder to store the results
OUT_PARENT_FOLDER = f"input/{INPUT_FILE_NAME}_ch{ch_idx}"

if EXPORT_MARKERS:
    # Create the folder to store the results if it does not exist
    if not os.path.exists(OUT_PARENT_FOLDER):
        os.makedirs(OUT_PARENT_FOLDER)

    np.save(f"{OUT_PARENT_FOLDER}/markers_{RIPPLE_BAND_FILENAME}_band.npy", final_ripple_markers)   # Save the data to a numpy file (not stored in git due to size)
    np.save(f"{OUT_PARENT_FOLDER}/markers_{FR_BAND_FILENAME}_band.npy", final_fr_markers)
    np.save(f"{OUT_PARENT_FOLDER}/markers_{BOTH_BAND_FILENAME}_band.npy", final_hfo_markers)

In [416]:
# To keep the same format as the original data, we will add an extra inner dimension to the final signal
final_ripple_signal = np.array([ripple_seeg_ch])
final_fr_signal = np.array([fr_seeg_ch])
final_hfo_signal = np.array([hfo_seeg_ch])

preview_np_array(final_ripple_signal, "Final Signal")
preview_np_array(final_fr_signal, "Final Signal")
preview_np_array(final_hfo_signal, "Final Signal")

Final Signal Shape: (1, 125056).
Preview: [[ 1.01688786e-04  1.48312419e-03  1.03505373e-02  4.59330006e-02
   1.44846460e-01 ...  2.79686384e+00  2.08877449e+00  1.23923651e+00
   3.91468362e-01 -3.35923047e-01]]
Final Signal Shape: (1, 125056).
Preview: [[ 0.0019679   0.01375153  0.02591011 -0.04557574 -0.24900832 ...
   0.40517923  0.04589511 -0.25397784 -0.85574955 -1.15821117]]
Final Signal Shape: (1, 125056).
Preview: [[ 0.08668457  0.81161464  3.1972572   6.36634517  4.56066843 ...
   1.46877805 -0.37909596 -0.66129324 -0.28660881 -0.62609222]]


In [417]:
EXPORT_SIGNAL = True
if EXPORT_SIGNAL:
    # Create the folder to store the results if it does not exist
    if not os.path.exists(OUT_PARENT_FOLDER):
        os.makedirs(OUT_PARENT_FOLDER)

    np.save(f"{OUT_PARENT_FOLDER}/{RIPPLE_BAND_FILENAME}_band.npy", final_ripple_signal)   # Save the data to a numpy file (not stored in git due to size)
    np.save(f"{OUT_PARENT_FOLDER}/{FR_BAND_FILENAME}_band.npy", final_fr_signal)
    np.save(f"{OUT_PARENT_FOLDER}/{BOTH_BAND_FILENAME}_band.npy", final_hfo_signal)